In [12]:
import os
import pytesseract
from pdf2image import convert_from_path
import re
from tqdm import tqdm  # Para mostrar el progreso
import gc  # Para la liberación de memoria

# Ruta al ejecutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Configuración del OCR (idioma español)
OCR_CONFIG = "--oem 3 --psm 6 -l spa"

# Carpeta con los PDF
folder_path = r"C:\Users\USUARIO\Downloads\Documentos VUR"  
output_txt_folder = "resultados_vur_txt"  # Carpeta donde se guardarán los archivos .txt

# Crear una carpeta de salida si no existe
if not os.path.exists(output_txt_folder):
    os.makedirs(output_txt_folder)

# Función para eliminar URLs y otros fragmentos no deseados
def limpiar_texto(texto):
    # Expresión regular para detectar URLs (http://, https://, www.)
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-f][0-9A-Fa-f]))+|www\.[a-zA-Z0-9-]+(?:\.[a-zA-Z]{2,})+"

    # Expresión regular para eliminar bloques de texto específicos:
    # - Encabezados:
    encabezado_pattern = r"\d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2} [ap]\.m\.\s*-\s*VUR"

    # - Pies de página con URL y formato específico
    pie_pagina_pattern = r"https://www\.vur\.gov\.co/portal/pages/vur/inicio\.jsf\?url=[^ ]+"

    # - Bloques no deseados específicos: "%%datosBasicosTierras" seguido por número y páginas
    bloque_no_deseado_pattern = r"%%datosBasicosTierras \d+/\d+"

    # Eliminar URLs
    texto = re.sub(url_pattern, "", texto)
    # Eliminar encabezados
    texto = re.sub(encabezado_pattern, "", texto)
    # Eliminar pies de página con URL
    texto = re.sub(pie_pagina_pattern, "", texto)
    # Eliminar bloques específicos de %%datosBasicosTierras
    texto = re.sub(bloque_no_deseado_pattern, "", texto)
    
    # Eliminar espacios al inicio y al final
    texto = texto.strip()
    
    return texto

# Función para extraer el texto de la imagen usando OCR
def extraer_texto_imagen(pdf_path):
    # Convertir el PDF a imágenes
    paginas = convert_from_path(pdf_path, 300)  # 300 es la resolución de la imagen
    
    # Extraer texto de cada página usando OCR
    texto_completo = ""
    for pagina in paginas:
        texto_completo += pytesseract.image_to_string(pagina, lang="spa", config="--psm 6")
    
    # Eliminar saltos de línea y hacer que el texto sea fluido
    texto_completo = texto_completo.replace("\n", " ")  # Reemplazar saltos de línea con espacio

    return texto_completo

# Función para procesar los archivos y guardar los resultados en archivos de texto
def procesar_y_guardar_texto(archivos, batch_size=10):
    # Usar tqdm para mostrar el progreso en lotes
    for i in tqdm(range(0, len(archivos), batch_size), desc="Procesando lotes de archivos", unit="lote"):
        batch = archivos[i:i+batch_size]

        for archivo in batch:
            try:
                # Abrir y procesar el PDF
                texto_pdf = extraer_texto_imagen(archivo)
                texto_limpio = limpiar_texto(texto_pdf)
                
                # Crear el nombre del archivo .txt basado en el nombre del archivo PDF
                nombre_archivo_txt = os.path.join(output_txt_folder, os.path.basename(archivo).replace(".pdf", ".txt"))
                
                # Guardar el texto limpio en el archivo .txt
                with open(nombre_archivo_txt, "w", encoding="utf-8") as f:
                    f.write(texto_limpio)

            except Exception as e:
                print(f"Error procesando {archivo}: {e}")

        # Liberar memoria entre lotes
        gc.collect()

# Obtener todos los archivos PDF en el directorio
archivos_pdf = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]

# Procesar los archivos y guardar el texto en archivos .txt
procesar_y_guardar_texto(archivos_pdf)

print("Proceso completado. Los textos extraídos han sido guardados en archivos .txt.")


Procesando lotes de archivos: 100%|██████████| 206/206 [5:17:11<00:00, 92.39s/lote]    

Proceso completado. Los textos extraídos han sido guardados en archivos .txt.


In [ ]:
import os
import csv
from tqdm import tqdm  # Barra de progreso
import gc

# Ruta a la carpeta que contiene los archivos .txt
input_txt_folder = "resultados_vur_txt"
output_csv_file = "datos_extraidos.csv"

# Función para extraer los datos entre las secciones indicadas
def extraer_dato(texto, inicio, fin):
    texto = texto.replace("N?", "N°").replace("N*", "N°").replace(" N ", "N°").replace("N°Matrícula", "N° Matrícula").replace("Inmobiliaría", "Inmobiliaria")
    start = texto.find(inicio)
    if start == -1:
        return None
    start += len(inicio)
    end = texto.find(fin, start)
    if end == -1:
        return texto[start:].strip()
    return texto[start:end].strip()

# Función para extraer los datos generales de cada archivo de texto
def extraer_datos(texto):
    # Extraer los datos de la consulta
    fecha = extraer_dato(texto, "Fecha:", "Hora:")
    hora = extraer_dato(texto, "Hora:", "No. Consulta:")
    consulta = extraer_dato(texto, "No. Consulta:", "N° Matrícula Inmobiliaria:")
    matricula = extraer_dato(texto, "N° Matrícula Inmobiliaria:", "Referencia Catastral:")
    referencia_catastral = extraer_dato(texto, "Referencia Catastral:", "Departamento:")
    departamento = extraer_dato(texto, "Departamento:", "Referencia Catastral Anterior:")
    referencia_catastral_anterior = extraer_dato(texto, "Referencia Catastral Anterior:", "Municipio:")
    municipio = extraer_dato(texto, "Municipio:", "Cédula Catastral:")
    cedula_catastral = extraer_dato(texto, "Cédula Catastral:", "Vereda:")
    vereda = extraer_dato(texto, "Vereda:", "Nupre:")
    nupre = extraer_dato(texto, "Nupre:", "Dirección Actual del Inmueble")
    direccion_actual = extraer_dato(texto, "Dirección Actual del Inmueble:", "Direcciones Anteriores")
    direcciones_anteriores = extraer_dato(texto, "Direcciones Anteriores:", "Determinacion:")
    determinacion = extraer_dato(texto, "Determinacion:", "Destinacion economica:")
    destinacion_economica = extraer_dato(texto, "Destinacion economica:", "Modalidad:")
    modalidad = extraer_dato(texto, "Modalidad:", "Fecha de Apertura del Folio:")
    fecha_apertura_folio = extraer_dato(texto, "Fecha de Apertura del Folio:", "Tipo de Instrumento:")
    tipo_instrumento = extraer_dato(texto, "Tipo de Instrumento:", "Fecha de Instrumento:")
    fecha_instrumento = extraer_dato(texto, "Fecha de Instrumento:", "Estado Folio:")
    estado_folio = extraer_dato(texto, "Estado Folio:", "Matrícula(s) Matriz:")
    matricula_matriz = extraer_dato(texto, "Matrícula(s) Matriz:", "Matrícula(s) Derivada(s):")
    matricula_derivada = extraer_dato(texto, "Matrícula(s) Derivada(s):", "Tipo de Predio:")
    tipo_predio = extraer_dato(texto, "Tipo de Predio:", "Alertas")
    complementaciones = extraer_dato(texto, "Complementaciones", "Cabidad y Linderos")
    cabidad_linderos = extraer_dato(texto, "Cabidad y Linderos", "Linderos Tecnicamente Definidos")
    linderos_tec_definidos = extraer_dato(texto, "Linderos Tecnicamente Definidos", "Area Y Coeficiente")

    # Crear un diccionario con todos los datos extraídos
    datos = {
        "Fecha": fecha,
        "Hora": hora,
        "No. Consulta": consulta,
        "Matrícula Inmobiliaria": matricula,
        "Referencia Catastral": referencia_catastral,
        "Referencia Catastral Anterior": referencia_catastral_anterior,
        "Cédula Catastral": cedula_catastral,
        "Departamento": departamento,
        "Municipio": municipio,
        "Vereda": vereda,
        "Nupre": nupre,
        "Dirección Actual del Inmueble": direccion_actual,
        "Direcciones Anteriores": direcciones_anteriores,
        "Determinacion": determinacion,
        "Destinacion economica": destinacion_economica,
        "Modalidad": modalidad,
        "Fecha de Apertura del Folio": fecha_apertura_folio,
        "Tipo de Instrumento": tipo_instrumento,
        "Fecha de Instrumento": fecha_instrumento,
        "Estado Folio": estado_folio,
        "Matrícula(s) Matriz": matricula_matriz,
        "Matrícula(s) Derivada(s)": matricula_derivada,
        "Tipo de Predio": tipo_predio,
        "Complementaciones": complementaciones,
        "Cabidad y Linderos": cabidad_linderos,
        "Linderos Técnicamente Definidos": linderos_tec_definidos,
    
    }
    return datos

# Función para procesar archivos en lotes
def procesar_en_lotes(archivos, batch_size=50):
    total_archivos = len(archivos)
    for i in range(0, total_archivos, batch_size):
        yield archivos[i:i + batch_size]

# Procesamiento principal
def procesar_archivos(input_folder, output_file, batch_size=50):
    archivos_txt = [f for f in os.listdir(input_folder) if f.endswith('.txt')]
    total_archivos = len(archivos_txt)

    with open(output_file, mode='a+', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=[
            "Fecha", "Hora", "No. Consulta", "Matrícula Inmobiliaria", "Referencia Catastral", 
            "Referencia Catastral Anterior", "Cédula Catastral", "Departamento", "Municipio", 
            "Vereda", "Nupre", "Dirección Actual del Inmueble", "Direcciones Anteriores", 
            "Determinacion", "Destinacion economica", "Modalidad", "Fecha de Apertura del Folio", 
            "Tipo de Instrumento", "Fecha de Instrumento", "Estado Folio", "Matrícula(s) Matriz", 
            "Matrícula(s) Derivada(s)", "Tipo de Predio", "Complementaciones", "Cabidad y Linderos", 
            "Linderos Técnicamente Definidos"], delimiter=';')

        # Escribe el encabezado solo si el archivo está vacío
        if file.tell() == 0:
            writer.writeheader()

        with tqdm(total=total_archivos, desc="Procesando archivos") as pbar:
            for batch in procesar_en_lotes(archivos_txt, batch_size):
                for filename in batch:
                    txt_path = os.path.join(input_folder, filename)
                    try:
                        with open(txt_path, 'r', encoding='utf-8') as f:
                            texto = f.read()  # Leer todo el archivo
                        datos_extraidos = extraer_datos(texto)
                        writer.writerow(datos_extraidos)
                    except Exception as e:
                        print(f"Error al procesar el archivo {filename}: {e}")
                pbar.update(len(batch))  # Actualiza la barra de progreso
                gc.collect()   # Recolecta basura para liberar memoria

# Ejecutar el procesamiento de archivos
procesar_archivos(input_txt_folder, output_csv_file)


Procesando archivos: 100%|██████████| 2056/2056 [00:03<00:00, 537.38it/s]
